# 🌳 Implementando uma Decision Tree do Zero

Este notebook mostra a implementação **passo a passo** de uma **árvore de decisão** em Python puro, sem o uso de bibliotecas prontas (como `scikit-learn`).

Nosso objetivo será entender **cada etapa** da construção da árvore, desde o cálculo da impureza até a classificação final, com explicações teóricas intercaladas.


## 1. O que é uma Árvore de Decisão?

Uma **árvore de decisão** é um modelo de aprendizado supervisionado que divide os dados em grupos cada vez mais homogêneos com base em perguntas (condições) sobre os atributos.

A ideia é encontrar, a cada divisão (nó da árvore), a **melhor condição** que separa as classes de forma mais “pura”.

Cada nó faz uma **pergunta** do tipo:

> “O atributo X é menor ou igual a um certo valor?”

E os dados são divididos entre os ramos **verdadeiro (esquerda)** e **falso (direita)**.


## 2. Estrutura da Classe `DecisionTree`

A classe `DecisionTree` será responsável por:
- **Treinar** a árvore com base nos dados (`fit`);
- **Classificar** novas amostras (`predict`);
- **Construir recursivamente** os nós da árvore;
- **Calcular a impureza** (Gini ou Entropia);
- **Selecionar** o melhor atributo e ponto de corte.


In [9]:
import numpy as np

class DecisionTree:
    def __init__(self, criterio="gini", max_depth=None):
        self.criterio = criterio
        self.max_depth = max_depth
        self.tree = None


## 3. Funções de Impureza: Gini e Entropia

O objetivo de cada divisão é **aumentar a pureza dos nós filhos**.

Para medir a pureza, usamos métricas como:

### Índice de Gini
$$
Gini = 1 - \sum_i p_i^2
$$

Onde $ p_i $ é a proporção de elementos da classe *i* no nó.

### Entropia
$$
H = - \sum_i p_i \log_2(p_i)
$$

A entropia mede o **grau de desordem** das classes — quanto maior, mais misturado o conjunto.


    def _gini(self, y):
        classes, counts = np.unique(y, return_counts=True)
        probs = counts / len(y)
        return 1 - np.sum(probs ** 2)

    def _entropy(self, y):
        classes, counts = np.unique(y, return_counts=True)
        probs = counts / len(y)
        return -np.sum(probs * np.log2(probs + 1e-9))  # evitar log(0)


In [10]:
    def _gini(self, y):
        classes, counts = np.unique(y, return_counts=True)
        probs = counts / len(y)
        return 1 - np.sum(probs ** 2)

    def _entropy(self, y):
        classes, counts = np.unique(y, return_counts=True)
        probs = counts / len(y)
        return -np.sum(probs * np.log2(probs + 1e-9))  # evitar log(0)


## 4. Dividindo os Dados

Cada divisão (split) separa os dados com base em um **atributo** e um **valor de corte**.

Por exemplo:
> “Se `altura <= 1.70`, vá para a esquerda; senão, vá para a direita.”


In [ ]:
    def _split(self, X, y, feature_index, threshold):
        esquerda = X[:, feature_index] <= threshold 
        direita = X[:, feature_index] > threshold 
        return X[esquerda], X[direita], y[esquerda], y[direita]


## 5. Escolhendo a Melhor Divisão

Precisamos testar **todas as possíveis divisões** e escolher a que **gera o maior ganho de informação** (ou seja, a maior redução da impureza total).


In [ ]:
def _best_split(self, X, y):
    # Inicializa a variável que rastreia o maior ganho de impureza encontrado até o momento.
    # O objetivo é maximizar este valor.
    melhor_ganho = -1 
    
    # Inicializa as variáveis que armazenarão a característica (feature) e o valor de corte (threshold)
    # que produzirem o 'melhor_ganho'.
    melhor_feature, melhor_threshold = None, None
    
    # Calcula a impureza do nó atual ANTES de qualquer divisão.
    # A métrica utilizada (Gini ou Entropia) depende do que foi configurado no atributo 'criterio'.
    impureza_atual = self._gini(y) if self.criterio == "gini" else self._entropy(y)

    # Obtém o número total de características (colunas) no conjunto de dados X.
    n_features = X.shape[1] 

    # --- INÍCIO DA BUSCA PELO MELHOR SPLIT ---
    
    # Loop principal: Itera sobre cada característica (coluna) do conjunto de dados.
    for feature in range(n_features): 
        # Obtém todos os valores únicos presentes na característica atual.
        # Estes valores serão testados como possíveis pontos de corte (thresholds).
        valores = np.unique(X[:, feature]) 
        
        # Loop interno: Itera sobre cada valor único como um potencial ponto de corte.
        for threshold in valores:
            # Chama a função auxiliar '_split' para dividir o conjunto de dados (X e y)
            # usando a 'feature' e o 'threshold' atuais.
            # X_esq/y_esq: Dados que satisfazem a condição (valor da feature <= threshold).
            # X_dir/y_dir: Dados que NÃO satisfazem a condição (valor da feature > threshold).
            X_esq, X_dir, y_esq, y_dir = self._split(X, y, feature, threshold)
            
            # Condição de parada: Se uma das divisões resultar em um nó vazio, a divisão é inválida.
            # Isso evita a criação de nós filhos vazios, que não contribuem para o ganho.
            if len(y_esq) == 0 or len(y_dir) == 0:
                continue

            # Calcula a proporção de exemplos que foram para o nó esquerdo ('esq').
            # p é o peso do nó esquerdo na impureza ponderada.
            p = len(y_esq) / len(y)
            
            # Calcula a impureza do nó esquerdo, usando o mesmo critério (Gini ou Entropia) do nó pai.
            impureza_esq = self._gini(y_esq) if self.criterio == "gini" else self._entropy(y_esq)
            
            # Calcula a impureza do nó direito.
            impureza_dir = self._gini(y_dir) if self.criterio == "gini" else self._entropy(y_dir)
            
            # Cálculo do GANHO DE IMPUREZA: 
            # É a diferença entre a impureza do nó pai e a soma ponderada das impurezas dos nós filhos.
            # Um valor maior significa uma redução mais eficaz na impureza.
            impureza_ponderada_depois = (p * impureza_esq + (1 - p) * impureza_dir)
            ganho = impureza_atual - impureza_ponderada_depois

            # --- ATUALIZAÇÃO DO MELHOR SPLIT ---
            
            # Se o ganho da divisão atual for maior que o melhor ganho registrado até agora,
            # armazena esta divisão como a melhor.
            if ganho > melhor_ganho:
                melhor_ganho = ganho
                melhor_feature = feature
                melhor_threshold = threshold

    # Retorna a característica e o ponto de corte que maximizaram o ganho de impureza.
    # Estas serão usadas para criar o próximo nó de teste na árvore.
    return melhor_feature, melhor_threshold

## 6. Construindo a Árvore Recursivamente

A construção da árvore é recursiva:
- Se o nó for “puro” (todas as classes iguais) ou atingir profundidade máxima → vira **folha**.
- Caso contrário → encontra o **melhor split**, divide e cria **nós filhos**.


In [ ]:
    def _build_tree(self, X, y, depth=0):
        classes, counts = np.unique(y, return_counts=True)
        node = {"depth": depth, "samples": len(y), "classes": dict(zip(classes, counts))}

        # Caso de parada
        if len(classes) == 1 or (self.max_depth is not None and depth >= self.max_depth):
            node["type"] = "leaf"
            node["prediction"] = classes[np.argmax(counts)]
            return node

        feature, threshold = self._best_split(X, y)
        if feature is None:
            node["type"] = "leaf"
            node["prediction"] = classes[np.argmax(counts)]
            return node

        node["type"] = "node"
        node["feature"] = feature
        node["threshold"] = threshold

        X_esq, X_dir, y_esq, y_dir = self._split(X, y, feature, threshold)
        node["left"] = self._build_tree(X_esq, y_esq, depth + 1)
        node["right"] = self._build_tree(X_dir, y_dir, depth + 1)

        return node


## 7. Treinamento e Predição

Agora, implementamos os métodos principais:
- `fit(X, y)`: constrói a árvore;
- `predict(X)`: percorre a árvore para classificar novas amostras.


In [ ]:
    def fit(self, X, y):
        self.tree = self._build_tree(X, y)

    def _predict_sample(self, x, node, caminho=None):
        if caminho is None:
            caminho = []

        if node["type"] == "leaf":
            caminho.append(f"→ Folha: classe = {node['prediction']}")
            return node["prediction"], caminho

        feature = node["feature"]
        threshold = node["threshold"]

        if x[feature] <= threshold:
            caminho.append(f"Se X[{feature}] <= {threshold}: seguir para esquerda")
            return self._predict_sample(x, node["left"], caminho)
        else:
            caminho.append(f"Se X[{feature}] > {threshold}: seguir para direita")
            return self._predict_sample(x, node["right"], caminho)

    def predict(self, X):
        preds, caminhos = [], []
        for x in X:
            pred, caminho = self._predict_sample(x, self.tree)
            preds.append(pred)
            caminhos.append(caminho)
        return np.array(preds), caminhos


## 8. Testando a Árvore com Dados Simples
Vamos testar com um conjunto de dados artificial.


In [8]:
X = np.array([
    [2.7, 2.5],
    [1.3, 1.5],
    [3.0, 3.5],
    [2.0, 1.0],
    [3.5, 0.5]
])
y = np.array([1, 0, 1, 0, 1])

arvore = DecisionTree(criterio="gini", max_depth=3)
arvore.fit(X, y)

preds, caminhos = arvore.predict(X)
for i, (p, c) in enumerate(zip(preds, caminhos)):
    print(f"Exemplo {i}: classe prevista = {p}")
    print("  Caminho de decisão:")
    for passo in c:
        print("   ", passo)
    print()


AttributeError: 'DecisionTree' object has no attribute 'fit'

## 9. Conclusão

Neste notebook, vimos **como uma árvore de decisão é construída do zero**, entendendo passo a passo:

- O cálculo da **impureza (Gini e Entropia)**;  
- O processo de **divisão ótima dos dados**;  
- A **recursão na construção da árvore**;  
- E, por fim, a **interpretação das decisões tomadas**.

Esse entendimento é essencial antes de usar implementações prontas como `DecisionTreeClassifier` do `scikit-learn`, pois nos permite compreender o *porquê* das decisões do modelo.
